In [1]:
from SensorDataset import SensorDataset
import tensorflow as tf
import numpy as np

In [2]:
sensor_dataset = SensorDataset("/home/rafaelpossas/dev/multimodal_dataset/sensor")

(200, 145, 19)


In [7]:
training_data_count = len(sensor_dataset.x_train)  # 7352 training series (with 50% overlap between each serie)
test_data_count = len(sensor_dataset.x_test)  # 2947 testing series
n_steps = len(sensor_dataset.x_train[0])  # 128 timesteps per series
n_input = len(sensor_dataset.x_train[0][0])  # 9 input parameters per timestep


# LSTM Neural Network's internal structure

n_hidden = 16 # Hidden layer num of features
n_classes = 20 # Total classes (should go up, or should go down)


# Training 

learning_rate = 0.0025
lambda_loss_amount = 0.0015
training_iters = training_data_count * 300  # Loop 300 times on the dataset
batch_size = 20
display_iter = 100  # To show test set accuracy during training

In [4]:
def LSTM_RNN(_X, _weights, _biases):
    # Function returns a tensorflow LSTM (RNN) artificial neural network from given parameters. 
    # Moreover, two LSTM cells are stacked which adds deepness to the neural network. 
    # Note, some code of this notebook is inspired from an slightly different 
    # RNN architecture used on another dataset: 
    # https://tensorhub.com/aymericdamien/tensorflow-rnn

    # (NOTE: This step could be greatly optimised by shaping the dataset once
    # input shape: (batch_size, n_steps, n_input)
    _X = tf.transpose(_X, [1, 0, 2])  # permute n_steps and batch_size
    # Reshape to prepare input to hidden activation
    _X = tf.reshape(_X, [-1, n_input]) 
    # new shape: (n_steps*batch_size, n_input)
    
    # Linear activation
    _X = tf.matmul(_X, _weights['hidden']) + _biases['hidden']
    # Split data because rnn cell needs a list of inputs for the RNN inner loop
    _X = tf.split(0, n_steps, _X) 
    # new shape: n_steps * (batch_size, n_hidden)

    # Define two stacked LSTM cells (two recurrent layers deep) with tensorflow
    lstm_cell_1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
    lstm_cell_2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
    lstm_cells = tf.nn.rnn_cell.MultiRNNCell([lstm_cell_1, lstm_cell_2], state_is_tuple=True)
    # Get LSTM cell output
    outputs, states = tf.nn.rnn(lstm_cells, _X, dtype=tf.float32)

    # Linear activation
    # Get inner loop last output
    return tf.matmul(outputs[-1], _weights['out']) + _biases['out']

In [5]:
def extract_batch_size(_train, step, batch_size):
    # Function to fetch a "batch_size" amount of data from "(X|y)_train" data. 
    
    shape = list(_train.shape)
    shape[0] = batch_size
    batch_s = np.empty(shape)

    for i in range(batch_size):
        # Loop index
        index = ((step-1)*batch_size + i) % len(_train)
        batch_s[i] = _train[index] 

    return batch_s

In [6]:
# Graph input/output
x = tf.placeholder(tf.float32, [None, n_steps, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])

# Graph weights
weights = {
    'hidden': tf.Variable(tf.random_normal([n_input, n_hidden])), # Hidden layer weights
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'hidden': tf.Variable(tf.random_normal([n_hidden])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

pred = LSTM_RNN(x, weights, biases)

# Loss, optimizer and evaluation
l2 = lambda_loss_amount * sum(
    tf.nn.l2_loss(tf_var) for tf_var in tf.trainable_variables()
) # L2 loss prevents this overkill neural network to overfit the data
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y)) + l2 # Softmax loss
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost) # Adam Optimizer

correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


In [8]:
# To keep track of training's performance
test_losses = []
test_accuracies = []
train_losses = []
train_accuracies = []

# Launch the graph
sess = tf.InteractiveSession(config=tf.ConfigProto(log_device_placement=True))
init = tf.initialize_all_variables()
sess.run(init)

# Perform Training steps with "batch_size" iterations at each loop
step = 1
while step * batch_size <= training_iters:
    batch_xs = extract_batch_size(sensor_dataset.x_train, step, batch_size)
    batch_ys = extract_batch_size(sensor_dataset.y_train, step, batch_size)

    # Fit training using batch data
    _, loss, acc = sess.run(
        [optimizer, cost, accuracy],
        feed_dict={
            x: batch_xs, 
            y: batch_ys
        }
    )
    train_losses.append(loss)
    train_accuracies.append(acc)
    
    # Evaluate network only at some steps for faster training: 
    if (step*batch_size % display_iter == 0) or (step == 1) or (step * batch_size > training_iters):
        
        # To not spam console, show training accuracy/loss in this "if"
        print "Training iter #" + str(step*batch_size) + \
              ":   Batch Loss = " + "{:.6f}".format(loss) + \
              ", Accuracy = {}".format(acc)
        
        # Evaluation on the test set (no learning made here - just evaluation for diagnosis)
        loss, acc = sess.run(
            [cost, accuracy], 
            feed_dict={
                x: sensor_dataset.x_test,
                y: sensor_dataset.y_test
            }
        )
        test_losses.append(loss)
        test_accuracies.append(acc)
        print "PERFORMANCE ON TEST SET: " + \
              "Batch Loss = {}".format(loss) + \
              ", Accuracy = {}".format(acc)

    step += 1

print "Optimization Finished!"

# Accuracy for test data

one_hot_predictions, accuracy, final_loss = sess.run(
    [pred, accuracy, cost],
    feed_dict={
        x: sensor_dataset.x_test,
        y: sensor_dataset.y_test
    }
)

test_losses.append(final_loss)
test_accuracies.append(accuracy)

print "FINAL RESULT: " + \
      "Batch Loss = {}".format(final_loss) + \
      ", Accuracy = {}".format(accuracy)


Training iter #20:   Batch Loss = 4.240007, Accuracy = 0.10000000149
PERFORMANCE ON TEST SET: Batch Loss = 4.72674274445, Accuracy = 0.0151515156031
Training iter #100:   Batch Loss = 4.073685, Accuracy = 0.0500000007451
PERFORMANCE ON TEST SET: Batch Loss = 4.52675199509, Accuracy = 0.0151515156031
Training iter #200:   Batch Loss = 3.729147, Accuracy = 0.10000000149
PERFORMANCE ON TEST SET: Batch Loss = 4.48767852783, Accuracy = 0.0303030312061
Training iter #300:   Batch Loss = 3.433275, Accuracy = 0.300000011921
PERFORMANCE ON TEST SET: Batch Loss = 4.41149568558, Accuracy = 0.0454545468092
Training iter #400:   Batch Loss = 3.851238, Accuracy = 0.0500000007451
PERFORMANCE ON TEST SET: Batch Loss = 4.38499069214, Accuracy = 0.0454545468092
Training iter #500:   Batch Loss = 3.596219, Accuracy = 0.15000000596
PERFORMANCE ON TEST SET: Batch Loss = 4.37555837631, Accuracy = 0.0454545468092
Training iter #600:   Batch Loss = 3.249743, Accuracy = 0.10000000149
PERFORMANCE ON TEST SET: B

KeyboardInterrupt: 

In [ ]:
`